In [ ]:
import pdbfixer as pf
from simtk.openmm.app import PDBFile

from pathlib import Path

from htmd.ui import *
from htmd import *

import matplotlib.pyplot as plt
import random
import numpy as np
import pandas as pd

from htmd.config import config
config(viewer='webgl')


from moleculekit.molecule import Molecule
from moleculekit.tools.voxeldescriptors import getVoxelDescriptors, viewVoxelFeatures
from moleculekit.tools.atomtyper import prepareProteinForAtomtyping
from moleculekit.smallmol.smallmol import SmallMol
from moleculekit.home import home
import os 

from moleculekit.molecule import Molecule
from openbabel import pybel

In [ ]:
from sklearn.preprocessing import OneHotEncoder 
import tensorflow as tf

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

from torch.optim import Adam
from torch.autograd import Variable

In [ ]:
current_directiry =  Path().absolute()

In [ ]:
s = "A   C   D   E   F   G   H   I   K   L   M   N   P   Q   R   S   T   V   W   Y".split()
s = np.array(s).reshape(-1, 1)

enc = OneHotEncoder()
encoded = enc.fit(s )

In [ ]:
data = [] 
X = []
Y = []

Nmax = 80 
nchannels = 8

for fn in Path(current_directiry).glob("*.pdb"):
    print(fn)
    prot = Molecule(str(fn),validateElements=False)
    
    prot.filter("protein")
    #print(len(list(prot.sequence()[key[0]]) ))
    #print(prot.sequence())
    
    key = list(prot.sequence() )    
    seq = np.array( list(prot.sequence()[key[0]]) )
    #print(len(seq), fn, seq.shape)
    
    if (len(seq)>Nmax):
        continue
    
    try:
        encoded_seq = encoded.transform(seq.reshape(-1, 1)).toarray().reshape(1,-1)
    except ValueError:
        print("loose", fn)
        continue
        
    #print(len(encoded_seq), encoded_seq.shape)
    label = np.zeros(Nmax*20)
    label[:len(seq)*20] = encoded_seq
    
    res_ids = np.unique(prot.resid)
    for i in res_ids:
        if(i>=0):
            prot.mutateResidue('resid ' + str(i), 'GLY')

        
    
    coo = prot.get('coords')
    c = np.mean(coo, axis=0)
    prot.moveBy(-c)
    

    try:
        prot = prepareProteinForAtomtyping(prot,  verbose=False)
    except RuntimeError:
        print("loose", fn)
        continue
    except ValueError:
        print("less bounds", fn)
        continue
        
        
        
    prot.filter("protein or ions")
    
    
    try:
        prot_vox, prot_centers, prot_N = getVoxelDescriptors(prot, boxsize=[30,30,30], center=[0,0,0])
    except ValueError:
        print("less bounds", fn)
 
    prot_vox_t = prot_vox.transpose().reshape([ nchannels, prot_N[0], prot_N[1], prot_N[2]])
    
    
    
    X.append(prot_vox_t)
    Y.append(label)
    
    print(prot_vox.shape, prot_N)
    #print(prot_vox.shape, prot_N)
    #break
    

In [ ]:
len(X)

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv3d(nchannels, 20, 5)  # (in_channels, out_channels, kernel_size)
        self.conv2 = nn.Conv3d(20, 20, 4)
        self.conv3 = nn.AvgPool3d(3)  #Conv3d(20, 20, 5)
        self.conv4 = nn.Conv3d(20, 25, 4) 
        self.conv5 = nn.Conv3d(25, 20*Nmax, 4) 
        self.ff1 = nn.Flatten()

    def forward(self, x):
        x1 = F.relu(self.conv1(x))
        x2= F.relu(self.conv2(x1))
        x3 = self.conv3(x2)
        x4 = self.conv4(x3)
        x5 = self.conv5(x4)
        x6 = self.ff1(x5)
        
        return F.sigmoid(x6)
        #return F.relu(self.conv4(x3))

In [ ]:
tensor_x = torch.Tensor(X) # transform to torch tensor
tensor_y = torch.Tensor(Y)

In [ ]:
#prots_x = torch.tensor( X )
tensor_y.shape

In [ ]:
from sklearn.model_selection import train_test_split

# create validation set
train_x, val_x, train_y, val_y = train_test_split( tensor_x, tensor_y, test_size = 0.1)
(train_x.shape, train_y.shape), (val_x.shape, val_y.shape)

In [ ]:
model = Model()
results = model.forward(tensor_x)
print(results.shape)

In [ ]:
# defining the model
model = Model()
# defining the optimizer
optimizer = Adam(model.parameters(), lr=0.07)
# defining the loss function
criterion = nn.CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    
print(model)

In [ ]:
 

def train(epoch):
    model.train()
    tr_loss = 0
 
    x_train, y_train = Variable(train_x), Variable(train_y)
 
    x_val, y_val = Variable(val_x), Variable(val_y)
 
    if torch.cuda.is_available():
        x_train = x_train.cuda()
        y_train = y_train.cuda()
        x_val = x_val.cuda()
        y_val = y_val.cuda()

 
    optimizer.zero_grad()
    
 
    output_train = model(x_train)
    output_val = model(x_val)
    #print(output_train.shape, output_val.shape, y_train.shape)
    # computing the training and validation loss
    #y_train = y_train.squeeze()
    loss_train = criterion(output_train.reshape(-1)  , y_train.reshape(-1)  )
    loss_val = criterion(output_val, y_val)
    train_losses.append(loss_train)
    val_losses.append(loss_val)

     
    loss_train.backward()
    optimizer.step()
    tr_loss = loss_train.item()
    #if epoch%2 == 0:
        # printing the validation loss
        #print('Epoch : ',epoch+1, '\t', 'loss :', loss_val)

In [ ]:
 
n_epochs = 25
 
train_losses = []
 
val_losses = []
 
for epoch in range(n_epochs):
    train(epoch)